In [1]:
import os
import sys
from pathlib import Path

sys.path.append('..')


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import pytorch_lightning as pl
import numpy as np

In [4]:
from baard.classifiers import MNIST_CNN
from baard.utils.torch_utils import dataloader2tensor, dataset2tensor

In [5]:
DATASET = 'MNIST'
EPS = 0.22  # Epsilon controls the adversarial perturbation.

PATH_ROOT = Path(os.getcwd()).absolute().parent
PATH_CHECKPOINT = os.path.join(PATH_ROOT, 'pretrained_clf', 'mnist_cnn.ckpt')
PATH_DATA_CLEAN = os.path.join(PATH_ROOT, 'results', 'exp1234', DATASET, 'AdvClean.n_100.pt')
PATH_DATA_ADV = os.path.join(PATH_ROOT, 'results', 'exp1234', DATASET, f'APGD.Linf.n_100.e_{EPS}.pt')
PATH_DETECTOR_DEV = os.path.join('temp', 'dev_lid_detector.lid')

# Parameters for development:
TINY_TEST_SIZE = 10
SEED_DEV = 0

pl.seed_everything(SEED_DEV)

print('PATH ROOT:', PATH_ROOT)
print('DATASET:', DATASET)
print('PATH_CHECKPOINT:', PATH_CHECKPOINT)

Global seed set to 0


PATH ROOT: /home/lukec/workspace/baard_v4
DATASET: MNIST
PATH_CHECKPOINT: /home/lukec/workspace/baard_v4/pretrained_clf/mnist_cnn.ckpt


In [6]:
my_model = MNIST_CNN.load_from_checkpoint(PATH_CHECKPOINT)

loader_train = my_model.train_dataloader()
loader_test = my_model.val_dataloader()

X_train, y_train = dataloader2tensor(loader_train)
X_test, y_test =dataloader2tensor(loader_test)

print('X_train:', X_train.size())
print('X_test:', X_test.size())

X_train: torch.Size([60000, 1, 28, 28])
X_test: torch.Size([10000, 1, 28, 28])


In [7]:
28 ** 2

784

In [8]:
latent_net = torch.nn.Sequential(
    *list(my_model.children())[:7]
)
latent_net.eval()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=9216, out_features=200, bias=True)
)

In [9]:
latent_features = latent_net(X_train[:TINY_TEST_SIZE])
print(latent_features.size())

torch.Size([10, 200])


In [10]:
print([f'{i:.2f}' for i in latent_features[0].detach().numpy()])

['-4.33', '-1.00', '2.11', '2.12', '-1.45', '-1.80', '-0.79', '-1.24', '2.02', '-1.66', '1.03', '-6.19', '-0.84', '-0.30', '1.17', '-2.68', '-0.72', '1.31', '0.02', '2.27', '-1.73', '-0.08', '-0.18', '0.44', '-0.28', '4.03', '-6.75', '-2.79', '6.30', '-0.08', '5.28', '-1.60', '3.15', '-0.11', '-0.33', '-6.50', '-7.51', '3.38', '-2.02', '-0.85', '-2.24', '4.24', '-3.67', '1.85', '1.47', '-2.31', '-0.73', '-4.32', '-2.03', '-0.15', '4.19', '-0.74', '0.24', '1.46', '-1.38', '-0.05', '0.20', '-0.75', '5.61', '-0.99', '4.01', '-4.20', '-6.47', '-11.91', '-6.38', '-0.30', '4.91', '1.00', '-1.41', '-8.04', '-3.03', '-0.25', '3.84', '4.51', '-3.93', '-9.84', '-1.67', '0.74', '0.09', '13.28', '-3.57', '-0.57', '-0.24', '0.27', '-0.39', '0.30', '0.96', '2.92', '-4.57', '-0.46', '-1.38', '-1.20', '-1.17', '3.26', '6.39', '-3.37', '1.01', '-7.35', '-3.42', '-1.77', '-2.76', '5.15', '-1.51', '2.73', '-4.38', '3.32', '-7.45', '12.59', '-0.65', '1.88', '0.75', '-0.61', '-0.18', '-0.09', '0.14', '-3.1

In [11]:
latent_feature_mean = latent_features.mean(0)
latent_feature_std = latent_features.std(0)

In [12]:
new_instance = latent_features[0]
z_score = (new_instance - latent_feature_mean) / latent_feature_std
z_score = z_score.detach().numpy()
z_max = np.abs(z_score).max()

z_max

2.443573

In [13]:
indices = torch.where(y_train == 0)[0]
y_train[indices]

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int16)